In [1]:
import pandas as pd

from sklearn.neighbors import LocalOutlierFactor
from sklearn.ensemble import IsolationForest
from sklearn.covariance import EllipticEnvelope

import statsmodels.regression.linear_model
from sklearn.ensemble import RandomForestRegressor
import numpy
import math

# counting null values

returns the same average value for columns and for rows

In [2]:
def avg_nulls(df):
    X = df.drop(['target'], axis=1)
    
    missing_columns = X.isnull().sum().mean()*100/X.shape[0]
    missing_rows = X.T.isnull().sum().mean()*100/X.shape[1]
    
    return missing_columns, missing_rows

## preprocessing fuction

df - indicators+target

options for deleting outliers:
* 'Local'
* 'IsolationForest'
* 'Elliptic'

returns: 
* number of nulls filled with median
* pandas.core.frame.DataFrame

In [3]:
def preprocessing(df, feature_not_missing_percentage=0.4, row_not_missing_percentage = 0.5, method_ol='Local'):
#deleting features with high rate of missing values
    feature_not_missing = feature_not_missing_percentage*df.shape[0]
    dataset = df.dropna(axis=1, thresh=feature_not_missing)
#deleting rows with high rate of missing values
    row_not_missing = row_not_missing_percentage*dataset.shape[1]
    dataset = dataset.dropna(axis=0, thresh=row_not_missing)
    
    nulls_count = sum(dataset.isnull().sum())
    
#filling nulls with medians
    for feature in list(dataset):
        dataset[feature].fillna(dataset[feature].median(), inplace=True)

#deleting outliers
    if method_ol == 'Local':
        #from sklearn.neighbors import LocalOutlierFactor
        LOF=LocalOutlierFactor(n_neighbors=10)
        df_with_lof = dataset.join(pd.DataFrame(LOF.fit_predict(dataset), index=dataset.index, columns=['lof']), how='left')
        outlier = df_with_lof.loc[df_with_lof['lof']!=1].index
    elif method_ol == 'IsolationForest':
        #from sklearn.ensemble import IsolationForest
        IF=IsolationForest(n_estimators=10)
        df_with_forest = dataset.join(pd.DataFrame(IF.fit_predict(dataset), index=dataset.index, columns=['isolation_forest']), how='left')
        outlier = df_with_forest.loc[df_with_forest['isolation_forest']!=1].index
    elif method_ol == 'Elliptic':
        #from sklearn.covariance import EllipticEnvelope
        Elliptic=EllipticEnvelope(contamination=0.1)
        df_with_ellip = dataset.join(pd.DataFrame(Elliptic.fit_predict(dataset), index=dataset.index, columns=['elliptic']), how='left')
        outlier = df_with_ellip.loc[df_with_ellip['elliptic']!=1].index
    
    dataset = dataset.drop(outlier)
    
    return nulls_count, dataset

# feature importances function

options:
* 'RandomForest'
* 'Linear'
* 'Correlation'

returns: pandas.core.series.Series

In [4]:
def feature_importances(dataset, method_fi='RandomForest'):
       
    Y = dataset['target']
    X = dataset.drop(['target'], axis=1)
    
    if method_fi == 'RandomForest':
        #from sklearn.ensemble import RandomForestRegressor
        #import numpy
        numpy.random.seed(0)
        result_rf = RandomForestRegressor(n_estimators=10)
        result_rf.fit(X,Y)
        feature_imp = pd.Series(result_rf.feature_importances_, index=X.columns)
    elif method_fi == 'Linear':
        #import statsmodels.regression.linear_model
        #import math
        result_ols = statsmodels.regression.linear_model.OLS(Y, X).fit()
        imp_ols = result_ols.params
        feature_imp = imp_ols/math.sqrt(sum(imp_ols*imp_ols))
    elif method_fi == 'Correlation':
        feature_imp = dataset.corr().drop(['target'], axis=0)['target']
        
    return feature_imp

If you don't need intermediate results you may use this:

In [5]:
def preprocessing_feature_importances(df, feature_not_missing_percentage=0.4, row_not_missing_percentage = 0.5, method_ol='Local', method_fi='RandomForest'):
    
#deleting features with high rate of missing values
    feature_not_missing = feature_not_missing_percentage*df.shape[0]
    dataset = df.dropna(axis=1, thresh=feature_not_missing)
#deleting rows with high rate of missing values
    row_not_missing = row_not_missing_percentage*dataset.shape[1]
    dataset = dataset.dropna(axis=0, thresh=row_not_missing)

#number of nulls filled with median
    nulls_count = sum(dataset.isnull().sum())
    
#filling nulls with medians
    for feature in list(dataset):
        dataset[feature].fillna(dataset[feature].median(), inplace=True)

#deleting outliers
    if method_ol == 'Local':
        #from sklearn.neighbors import LocalOutlierFactor
        LOF=LocalOutlierFactor(n_neighbors=10)
        df_with_lof = dataset.join(pd.DataFrame(LOF.fit_predict(dataset), index=dataset.index, columns=['lof']), how='left')
        outlier = df_with_lof.loc[df_with_lof['lof']!=1].index
    elif method_ol == 'IsolationForest':
        #from sklearn.ensemble import IsolationForest
        IF=IsolationForest(n_estimators=10)
        df_with_forest = dataset.join(pd.DataFrame(IF.fit_predict(dataset), index=dataset.index, columns=['isolation_forest']), how='left')
        outlier = df_with_forest.loc[df_with_forest['isolation_forest']!=1].index
    elif method_ol == 'Elliptic':
        #from sklearn.covariance import EllipticEnvelope
        Elliptic=EllipticEnvelope(contamination=0.1)
        df_with_ellip = dataset.join(pd.DataFrame(Elliptic.fit_predict(dataset), index=dataset.index, columns=['elliptic']), how='left')
        outlier = df_with_ellip.loc[df_with_ellip['elliptic']!=1].index
    
    dataset = dataset.drop(outlier)
    
#calculating feature importances
    Y = dataset['target']
    X = dataset.drop(['target'], axis=1)
    
    if method_fi == 'RandomForest':
        #from sklearn.ensemble import RandomForestRegressor
        #import numpy
        numpy.random.seed(0)
        result_rf = RandomForestRegressor(n_estimators=10)
        result_rf.fit(X,Y)
        feature_imp = pd.Series(result_rf.feature_importances_, index=X.columns)
    elif method_fi == 'Linear':
        #import statsmodels.regression.linear_model
        #import math
        result_ols = statsmodels.regression.linear_model.OLS(Y, X).fit()
        imp_ols = result_ols.params
        feature_imp = imp_ols/math.sqrt(sum(imp_ols*imp_ols))
    elif method_fi == 'Correlation':
        feature_imp = dataset.corr().drop(['target'], axis=0)['target']
        
    return nulls_count, feature_imp

# Test

In [6]:
indicators_2003 = pd.read_json('data_influence/indicators/ind2003.json').pivot(index='country_doc_id_id',columns='eng_name',values='value')
indicators_2004 = pd.read_json('data_influence/indicators/ind2004.json').pivot(index='country_doc_id_id',columns='eng_name',values='value')
indicators_2005 = pd.read_json('data_influence/indicators/ind2005.json').pivot(index='country_doc_id_id',columns='eng_name',values='value')
indicators_2006 = pd.read_json('data_influence/indicators/ind2006.json').pivot(index='country_doc_id_id',columns='eng_name',values='value')
indicators_2007 = pd.read_json('data_influence/indicators/ind2007.json').pivot(index='country_doc_id_id',columns='eng_name',values='value')
indicators_2008 = pd.read_json('data_influence/indicators/ind2008.json').pivot(index='country_doc_id_id',columns='eng_name',values='value')
indicators_2009 = pd.read_json('data_influence/indicators/ind2009.json').pivot(index='country_doc_id_id',columns='eng_name',values='value')
indicators_2010 = pd.read_json('data_influence/indicators/ind2010.json').pivot(index='country_doc_id_id',columns='eng_name',values='value')
indicators_2011 = pd.read_json('data_influence/indicators/ind2011.json').pivot(index='country_doc_id_id',columns='eng_name',values='value')
indicators_2012 = pd.read_json('data_influence/indicators/ind2012.json').pivot(index='country_doc_id_id',columns='eng_name',values='value')
indicators_2013 = pd.read_json('data_influence/indicators/ind2013.json').pivot(index='country_doc_id_id',columns='eng_name',values='value')
indicators_2014 = pd.read_json('data_influence/indicators/ind2014.json').pivot(index='country_doc_id_id',columns='eng_name',values='value')
indicators_2015 = pd.read_json('data_influence/indicators/ind2015.json').pivot(index='country_doc_id_id',columns='eng_name',values='value')
indicators_2016 = pd.read_json('data_influence/indicators/ind2016.json').pivot(index='country_doc_id_id',columns='eng_name',values='value')

indicators = {'indicators_2003': indicators_2003, 'indicators_2004': indicators_2004,
            'indicators_2005': indicators_2005, 'indicators_2006': indicators_2006,
            'indicators_2007': indicators_2007, 'indicators_2008': indicators_2008,
            'indicators_2009': indicators_2009, 'indicators_2010': indicators_2010, 
             'indicators_2011': indicators_2011, 'indicators_2012': indicators_2012, 
             'indicators_2013': indicators_2013, 'indicators_2014': indicators_2014,
             'indicators_2015': indicators_2015, 'indicators_2016': indicators_2016}

In [7]:
bribery_2013 = pd.read_json('data_influence/crimes/bribery2013.json').set_index('country_doc_id_id')
bribery_2014 = pd.read_json('data_influence/crimes/bribery2014.json').set_index('country_doc_id_id')
bribery_2015 = pd.read_json('data_influence/crimes/bribery2015.json').set_index('country_doc_id_id')
bribery_2016 = pd.read_json('data_influence/crimes/bribery2016.json').set_index('country_doc_id_id')
corruption_2013 = pd.read_json('data_influence/crimes/corruption2013.json').set_index('country_doc_id_id')
corruption_2014 = pd.read_json('data_influence/crimes/corruption2014.json').set_index('country_doc_id_id')
corruption_2015 = pd.read_json('data_influence/crimes/corruption2015.json').set_index('country_doc_id_id')
corruption_2016 = pd.read_json('data_influence/crimes/corruption2016.json').set_index('country_doc_id_id')
kidnapping_2003 = pd.read_json('data_influence/crimes/kidnapping2003.json').set_index('country_doc_id_id')
kidnapping_2004 = pd.read_json('data_influence/crimes/kidnapping2004.json').set_index('country_doc_id_id')
kidnapping_2005 = pd.read_json('data_influence/crimes/kidnapping2005.json').set_index('country_doc_id_id')
kidnapping_2006 = pd.read_json('data_influence/crimes/kidnapping2006.json').set_index('country_doc_id_id')
kidnapping_2007 = pd.read_json('data_influence/crimes/kidnapping2007.json').set_index('country_doc_id_id')
kidnapping_2008 = pd.read_json('data_influence/crimes/kidnapping2008.json').set_index('country_doc_id_id')
kidnapping_2009 = pd.read_json('data_influence/crimes/kidnapping2009.json').set_index('country_doc_id_id')
kidnapping_2010 = pd.read_json('data_influence/crimes/kidnapping2010.json').set_index('country_doc_id_id')
kidnapping_2011 = pd.read_json('data_influence/crimes/kidnapping2011.json').set_index('country_doc_id_id')
kidnapping_2012 = pd.read_json('data_influence/crimes/kidnapping2012.json').set_index('country_doc_id_id')
kidnapping_2013 = pd.read_json('data_influence/crimes/kidnapping2013.json').set_index('country_doc_id_id')
kidnapping_2014 = pd.read_json('data_influence/crimes/kidnapping2014.json').set_index('country_doc_id_id')
kidnapping_2015 = pd.read_json('data_influence/crimes/kidnapping2015.json').set_index('country_doc_id_id')
kidnapping_2016 = pd.read_json('data_influence/crimes/kidnapping2016.json').set_index('country_doc_id_id')
serious_assault_2003 = pd.read_json('data_influence/crimes/serious_assault2003.json').set_index('country_doc_id_id')
serious_assault_2004 = pd.read_json('data_influence/crimes/serious_assault2004.json').set_index('country_doc_id_id')
serious_assault_2005 = pd.read_json('data_influence/crimes/serious_assault2005.json').set_index('country_doc_id_id')
serious_assault_2006 = pd.read_json('data_influence/crimes/serious_assault2006.json').set_index('country_doc_id_id')
serious_assault_2007 = pd.read_json('data_influence/crimes/serious_assault2007.json').set_index('country_doc_id_id')
serious_assault_2008 = pd.read_json('data_influence/crimes/serious_assault2008.json').set_index('country_doc_id_id')
serious_assault_2009 = pd.read_json('data_influence/crimes/serious_assault2009.json').set_index('country_doc_id_id')
serious_assault_2010 = pd.read_json('data_influence/crimes/serious_assault2010.json').set_index('country_doc_id_id')
serious_assault_2011 = pd.read_json('data_influence/crimes/serious_assault2011.json').set_index('country_doc_id_id')
serious_assault_2012 = pd.read_json('data_influence/crimes/serious_assault2012.json').set_index('country_doc_id_id')
serious_assault_2013 = pd.read_json('data_influence/crimes/serious_assault2013.json').set_index('country_doc_id_id')
serious_assault_2014 = pd.read_json('data_influence/crimes/serious_assault2014.json').set_index('country_doc_id_id')
serious_assault_2015 = pd.read_json('data_influence/crimes/serious_assault2015.json').set_index('country_doc_id_id')
serious_assault_2016 = pd.read_json('data_influence/crimes/serious_assault2016.json').set_index('country_doc_id_id')
theft_2003 = pd.read_json('data_influence/crimes/theft2003.json').set_index('country_doc_id_id')
theft_2004 = pd.read_json('data_influence/crimes/theft2004.json').set_index('country_doc_id_id')
theft_2005 = pd.read_json('data_influence/crimes/theft2005.json').set_index('country_doc_id_id')
theft_2006 = pd.read_json('data_influence/crimes/theft2006.json').set_index('country_doc_id_id')
theft_2007 = pd.read_json('data_influence/crimes/theft2007.json').set_index('country_doc_id_id')
theft_2008 = pd.read_json('data_influence/crimes/theft2008.json').set_index('country_doc_id_id')
theft_2009 = pd.read_json('data_influence/crimes/theft2009.json').set_index('country_doc_id_id')
theft_2010 = pd.read_json('data_influence/crimes/theft2010.json').set_index('country_doc_id_id')
theft_2011 = pd.read_json('data_influence/crimes/theft2011.json').set_index('country_doc_id_id')
theft_2012 = pd.read_json('data_influence/crimes/theft2012.json').set_index('country_doc_id_id')
theft_2013 = pd.read_json('data_influence/crimes/theft2013.json').set_index('country_doc_id_id')
theft_2014 = pd.read_json('data_influence/crimes/theft2014.json').set_index('country_doc_id_id')
theft_2015 = pd.read_json('data_influence/crimes/theft2015.json').set_index('country_doc_id_id')
theft_2016 = pd.read_json('data_influence/crimes/theft2016.json').set_index('country_doc_id_id')

targets = {'bribery_2013':bribery_2013, 'bribery_2014':bribery_2014, 'bribery_2015': bribery_2015, 
          'bribery_2016':bribery_2016, 'corruption_2013':corruption_2013, 'corruption_2014':corruption_2014, 
          'corruption_2015':corruption_2015, 'corruption_2016':corruption_2016, 'kidnapping_2003':kidnapping_2003, 
          'kidnapping_2004':kidnapping_2004, 'kidnapping_2005':kidnapping_2005, 'kidnapping_2006':kidnapping_2006,
          'kidnapping_2007':kidnapping_2007, 'kidnapping_2008':kidnapping_2008, 'kidnapping_2009':kidnapping_2009, 
          'kidnapping_2010':kidnapping_2010, 'kidnapping_2011':kidnapping_2011, 'kidnapping_2012':kidnapping_2012, 
          'kidnapping_2013':kidnapping_2013, 'kidnapping_2014':kidnapping_2014, 'kidnapping_2015':kidnapping_2015,
          'kidnapping_2016':kidnapping_2016, 'serious_assault_2003':serious_assault_2003, 
          'serious_assault_2004':serious_assault_2004, 'serious_assault_2005':serious_assault_2005, 
          'serious_assault_2006':serious_assault_2006, 'serious_assault_2007':serious_assault_2007, 
          'serious_assault_2008':serious_assault_2008, 'serious_assault_2009':serious_assault_2009,
          'serious_assault_2010':serious_assault_2010, 'serious_assault_2011':serious_assault_2011, 
          'serious_assault_2012':serious_assault_2012, 'serious_assault_2013':serious_assault_2013, 
          'serious_assault_2014':serious_assault_2014, 'serious_assault_2015':serious_assault_2015, 
          'serious_assault_2016':serious_assault_2016, 'theft_2003':theft_2003, 'theft_2004':theft_2004, 
          'theft_2005':theft_2005, 'theft_2006':theft_2006, 'theft_2007':theft_2007, 'theft_2008':theft_2008, 
          'theft_2009':theft_2009, 'theft_2010':theft_2010, 'theft_2011':theft_2011, 'theft_2012':theft_2012, 
          'theft_2013':theft_2013, 'theft_2014':theft_2014, 'theft_2015':theft_2015, 'theft_2016':theft_2016}

In [12]:
data = indicators_2014.merge(serious_assault_2014,on='country_doc_id_id')
avg_nulls(data)

(33.54908306364617, 33.54908306364617)

In [13]:
preprocessing_feature_importances(data)

(146,
 GDP per capita (current US$)                                                                                                  0.165590
 GINI index (World Bank estimate)                                                                                              0.095942
 Government expenditure on education, total (% of GDP)                                                                         0.050200
 Population growth (annual %)                                                                                                  0.087824
 Share of youth not in education, employment or training, total (% of youth population)                                        0.105291
 UIS: Percentage Of Population Age 25+ With At Least A Completed Bachelor's Or Equivalent Degree (ISCED 6 Or Higher). Total    0.235460
 Unemployment, total (% of total labor force) (national estimate)                                                              0.259693
 dtype: float64)

# TRASH

# total nulls in data

In [ ]:
null_count = pd.DataFrame(index = indicators_2010.isnull().sum().index)
null_count['2003'] = indicators_2003.isnull().sum()
null_count['2004'] = indicators_2004.isnull().sum()
null_count['2005'] = indicators_2005.isnull().sum()
null_count['2006'] = indicators_2006.isnull().sum()
null_count['2007'] = indicators_2007.isnull().sum()
null_count['2008'] = indicators_2008.isnull().sum()
null_count['2009'] = indicators_2009.isnull().sum()
null_count['2010'] = indicators_2010.isnull().sum()
null_count['2011'] = indicators_2011.isnull().sum()
null_count['2012'] = indicators_2012.isnull().sum()
null_count['2013'] = indicators_2013.isnull().sum()
null_count['2014'] = indicators_2014.isnull().sum()
null_count['2015'] = indicators_2015.isnull().sum()
null_count['2016'] = indicators_2016.isnull().sum()
null_count

In time period since 2012 till 2015 there are 216 values in total. In other cases - 217.

Theft data is the fullest, so let's take it for example.

In [11]:
data = indicators_2010.merge(theft_2010,on='country_doc_id_id')
data

,Barro-Lee: Percentage Of Population Age 15+ With No Education,GDP per capita (current US$),GINI index (World Bank estimate),"Government expenditure on education, total (% of GDP)","Literacy rate, adult total (% of people ages 15 and above)",Population growth (annual %),"Share of youth not in education, employment or training, total (% of youth population)","Total alcohol consumption per capita (liters of pure alcohol, projected estimates, 15+ years of age)",UIS: Percentage Of Population Age 25+ With At Least A Completed Bachelor's Or Equivalent Degree (ISCED 6 Or Higher). Total,UIS: Percentage Of Population Age 25+ With No Schooling. Total,"Unemployment, total (% of total labor force) (national estimate)",target
country_doc_id_id,,,,,,,,,,,,
ALB,3.28,4094.362119,NaN,NaN,NaN,-0.496462,29.501900,7.9,NaN,NaN,14.086000,3978
AND,NaN,39736.354063,NaN,3.05742,NaN,-0.016577,NaN,11.4,NaN,NaN,NaN,1117
ARM,0.60,3218.372707,30.0,3.24900,NaN,-0.373712,NaN,5.6,NaN,NaN,19.007999,3664
AUS,0.72,52022.125596,34.7,5.55006,NaN,1.555490,11.320000,12.5,NaN,0.13187,5.211300,477265
AUT,1.29,46858.043273,30.3,5.70048,NaN,0.240394,7.420000,12.0,NaN,NaN,4.820000,155857
...,...,...,...,...,...,...,...,...,...,...,...,...
UKR,1.25,2965.142365,24.8,NaN,NaN,-0.397285,NaN,14.3,NaN,NaN,8.100000,254755
URY,1.34,11992.016627,44.5,NaN,98.07271,0.286155,18.123600,9.2,NaN,1.62115,7.158200,95269
USA,0.35,48466.823375,40.3,NaN,NaN,0.827846,19.884899,9.3,NaN,0.39338,9.633400,6204601


In [8]:
list(data)

['Barro-Lee: Percentage Of Population Age 15+ With No Education',
 'GDP per capita (current US$)',
 'GINI index (World Bank estimate)',
 'Government expenditure on education, total (% of GDP)',
 'Literacy rate, adult total (% of people ages 15 and above)',
 'Population growth (annual %)',
 'Share of youth not in education, employment or training, total (% of youth population)',
 'Total alcohol consumption per capita (liters of pure alcohol, projected estimates, 15+ years of age)',
 "UIS: Percentage Of Population Age 25+ With At Least A Completed Bachelor's Or Equivalent Degree (ISCED 6 Or Higher). Total",
 'UIS: Percentage Of Population Age 25+ With No Schooling. Total',
 'Unemployment, total (% of total labor force) (national estimate)',
 'target']

# counting nulls

In [39]:
def avg_nulls(df):
    X = df.drop(['target'], axis=1)
    
    missing_columns = X.isnull().sum().mean()*100/X.shape[0]
    missing_rows = X.T.isnull().sum().mean()*100/X.shape[1]
    
    return missing_columns, missing_rows
    

In [40]:
avg_nulls(data)

(34.361968306922435, 34.361968306922435)

# handling missing values

As data for 2010 contains all features, I will use this DataFrame for testing.

In [ ]:
data

In [ ]:
mean_median = pd.DataFrame(data={'means':data.mean(),
                                 'medians': data.median()})
mean_median

### deleting features with a high rate of missing values

In [ ]:
feature_not_missing_percentage=0.6
feature_missing = feature_not_missing_percentage*data.shape[0]
feature_missing

In [ ]:
indicators_without_nulls = data.dropna(axis=1, thresh=feature_missing)
#indicators_without_nulls

In [ ]:
indicators_without_nulls.isnull().sum()

### deleting rows with a high rate of missing values

In [ ]:
row_not_missing_percentage = 0.5
row_missing = row_not_missing_percentage*indicators_without_nulls.shape[1]
row_missing

In [ ]:
indicators_without_nulls = indicators_without_nulls.dropna(axis=0, thresh=row_missing)
#indicators_without_nulls

In [ ]:
indicators_without_nulls.isnull().sum()

### filling nulls with medians

In [ ]:
mean_median = pd.DataFrame(data={'means':indicators_without_nulls.mean(),
                                 'medians': indicators_without_nulls.median()})
mean_median

In [ ]:
for feature in list(indicators_without_nulls):
    indicators_without_nulls[feature].fillna(indicators_without_nulls[feature].median(), inplace=True)

indicators_without_nulls

In [ ]:
indicators_without_nulls.isnull().sum()

In [47]:
def handling_nulls(df):
#deleting features with high rate of missing values
    feature_not_missing_percentage=0.3
    feature_not_missing = feature_not_missing_percentage*df.shape[0]
    without_nulls = df.dropna(axis=1, thresh=feature_not_missing)
#deleting rows with high rate of missing values
    row_not_missing_percentage = 0.5
    row_not_missing = row_not_missing_percentage*without_nulls.shape[1]
    without_nulls = without_nulls.dropna(axis=0, thresh=row_not_missing)
    
    nulls_count = sum(without_nulls.isnull().sum())
    
#filling nulls with medians
    for feature in list(without_nulls):
        without_nulls[feature].fillna(without_nulls[feature].median(), inplace=True)
    return nulls_count, without_nulls

In [48]:
handling_nulls(data)
#data

(198,
                    Barro-Lee: Percentage Of Population Age 15+ With No Education  \
 country_doc_id_id                                                                  
 ALB                                                             3.28               
 AND                                                             2.84               
 ARM                                                             0.60               
 AUS                                                             0.72               
 AUT                                                             1.29               
 ...                                                              ...               
 UGA                                                            12.18               
 UKR                                                             1.25               
 URY                                                             1.34               
 USA                                                       

# deleting outliers

https://scikit-learn.org/stable/modules/outlier_detection.html 

https://scikit-learn.org/stable/auto_examples/neighbors/plot_lof_outlier_detection.html#sphx-glr-auto-examples-neighbors-plot-lof-outlier-detection-py 

In [ ]:
from sklearn.neighbors import LocalOutlierFactor

In [ ]:
LOF=LocalOutlierFactor(n_neighbors=2)

In [ ]:
df_with_lof = data.join(pd.DataFrame(LOF.fit_predict(data), index=data.index, columns=['lof']), how='left')

In [ ]:
df_with_lof.loc[df_with_lof['lof']!=1].index

In [ ]:
from sklearn.ensemble import IsolationForest

In [ ]:
IF=IsolationForest(n_estimators=10)

In [ ]:
df_with_forest = data.join(pd.DataFrame(IF.fit_predict(data), index=data.index, columns=['isolation_forest']), how='left')

In [ ]:
df_with_forest.loc[df_with_forest['isolation_forest']!=1].index

In [ ]:
from sklearn.covariance import EllipticEnvelope

In [ ]:
Elliptic=EllipticEnvelope(contamination=0.5)

In [ ]:
df_with_ellip = data.join(pd.DataFrame(Elliptic.fit_predict(data), index=data.index, columns=['elliptic']), how='left')

In [ ]:
df_with_ellip.loc[df_with_ellip['elliptic']!=1].index

In [ ]:
def outliers_fit(df, method='Local'):
    dataset = df.copy()
    
    if method == 'Local':
        from sklearn.neighbors import LocalOutlierFactor
        LOF=LocalOutlierFactor(n_neighbors=10)
        df_with_lof = dataset.join(pd.DataFrame(LOF.fit_predict(dataset), index=dataset.index, columns=['lof']), how='left')
        outlier = df_with_lof.loc[df_with_lof['lof']!=1].index
    elif method == 'IsolationForest':
        from sklearn.ensemble import IsolationForest
        IF=IsolationForest(n_estimators=10)
        df_with_forest = dataset.join(pd.DataFrame(IF.fit_predict(dataset), index=dataset.index, columns=['isolation_forest']), how='left')
        outlier = df_with_forest.loc[df_with_forest['isolation_forest']!=1].index
    elif method == 'Elliptic':
        from sklearn.covariance import EllipticEnvelope
        Elliptic=EllipticEnvelope(contamination=0.1)
        df_with_ellip = dataset.join(pd.DataFrame(Elliptic.fit_predict(dataset), index=dataset.index, columns=['elliptic']), how='left')
        outlier = df_with_ellip.loc[df_with_ellip['elliptic']!=1].index
    
    dataset = dataset.drop(outlier)
    return dataset

# preprocessing

In [49]:
def preprocessing(df, feature_not_missing_percentage=0.4, row_not_missing_percentage = 0.5, method='Local'):
#deleting features with high rate of missing values
    feature_not_missing = feature_not_missing_percentage*df.shape[0]
    without_nulls = df.dropna(axis=1, thresh=feature_not_missing)
#deleting rows with high rate of missing values
    row_not_missing = row_not_missing_percentage*without_nulls.shape[1]
    without_nulls = without_nulls.dropna(axis=0, thresh=row_not_missing)
    
    nulls_count = sum(without_nulls.isnull().sum())
    
#filling nulls with medians
    for feature in list(without_nulls):
        without_nulls[feature].fillna(without_nulls[feature].median(), inplace=True)

#deleting outliers
    dataset = without_nulls.copy()
    
    if method == 'Local':
        from sklearn.neighbors import LocalOutlierFactor
        LOF=LocalOutlierFactor(n_neighbors=10)
        df_with_lof = dataset.join(pd.DataFrame(LOF.fit_predict(dataset), index=dataset.index, columns=['lof']), how='left')
        outlier = df_with_lof.loc[df_with_lof['lof']!=1].index
    elif method == 'IsolationForest':
        from sklearn.ensemble import IsolationForest
        IF=IsolationForest(n_estimators=10)
        df_with_forest = dataset.join(pd.DataFrame(IF.fit_predict(dataset), index=dataset.index, columns=['isolation_forest']), how='left')
        outlier = df_with_forest.loc[df_with_forest['isolation_forest']!=1].index
    elif method == 'Elliptic':
        from sklearn.covariance import EllipticEnvelope
        Elliptic=EllipticEnvelope(contamination=0.1)
        df_with_ellip = dataset.join(pd.DataFrame(Elliptic.fit_predict(dataset), index=dataset.index, columns=['elliptic']), how='left')
        outlier = df_with_ellip.loc[df_with_ellip['elliptic']!=1].index
    
    dataset = dataset.drop(outlier)
    
    return nulls_count, dataset

In [50]:
nulls, data = preprocessing(data)
data

,Barro-Lee: Percentage Of Population Age 15+ With No Education,GDP per capita (current US$),GINI index (World Bank estimate),"Government expenditure on education, total (% of GDP)",Population growth (annual %),"Share of youth not in education, employment or training, total (% of youth population)","Total alcohol consumption per capita (liters of pure alcohol, projected estimates, 15+ years of age)","Unemployment, total (% of total labor force) (national estimate)",target
country_doc_id_id,,,,,,,,,
ALB,3.28,4094.362119,33.15,4.830355,-0.496462,29.501900,7.9,14.086000,3978
ARM,0.60,3218.372707,30.00,3.249000,-0.373712,15.436100,5.6,19.007999,3664
AUT,1.29,46858.043273,30.30,5.700480,0.240394,7.420000,12.0,4.820000,155857
AZE,2.84,5842.805784,33.15,2.780620,1.189788,15.436100,2.9,5.630000,3868
BEL,4.67,44141.878142,28.40,6.407990,0.913639,10.860000,11.4,8.292500,228355
...,...,...,...,...,...,...,...,...,...
TUR,9.18,10672.389245,38.80,4.830355,1.400093,32.290001,2.2,10.659700,179586
UGA,12.18,622.498846,33.15,2.389010,3.186619,15.436100,13.2,7.407600,23429
UKR,1.25,2965.142365,24.80,4.830355,-0.397285,15.436100,14.3,8.100000,254755


In [51]:
nulls

131

# feature importances

In [7]:
import matplotlib.pyplot as plt
%matplotlib inline

In [8]:
Y = data['target']
X = data.drop(['target'], axis=1)

### linear regression

In [9]:
import statsmodels.formula.api as sm

In [ ]:
result_ols = sm.OLS(Y, X).fit() 
result_ols.summary()

In [ ]:
imp_ols = result_ols.params
imp_ols

In [ ]:
imp_ols_r = result_ols.params
mx = max(imp_ols_r)
mn = min(imp_ols_r)
imp_ols_arr = []
for i in range(len(imp_ols_r)):
    imp_ols_arr.append((imp_ols_r[i]-mn)/(mx-mn))
imp_ols_arr

In [ ]:
pd.Series(imp_ols_arr, index=X.columns).sort_values().plot(kind='barh', figsize=(12, 8))

In [ ]:
import math

In [ ]:
imp_ols = imp_ols/math.sqrt(sum(imp_ols*imp_ols))
imp_ols

In [ ]:
imp_ols = pd.DataFrame(imp_ols, index=imp_ols.index, columns=['importance'])
imp_ols.sort_values('importance').plot(kind='barh', figsize=(12, 8))

In [ ]:
imp_ols = pd.DataFrame(result_ols.params, index=result_ols.params.index, columns=['importance'])
imp_ols.sort_values('importance').plot(kind='barh', figsize=(12, 8))

### random forest regressor

In [ ]:
from sklearn.ensemble import RandomForestRegressor
import numpy as np

In [ ]:
np.random.seed(0)

In [ ]:
result_rf = RandomForestRegressor(n_estimators=10)

In [ ]:
result_rf.fit(X,Y)

In [ ]:
result_rf.feature_importances_ 

In [ ]:
imp_rf = pd.Series(result_rf.feature_importances_, index=X.columns)
imp_rf

In [ ]:
imp_rf = pd.DataFrame(result_rf.feature_importances_, index=X.columns, columns=['importance'])
imp_rf.sort_values('importance').plot(kind='barh', figsize=(12, 8))

# correlation

In [ ]:
data.corr().drop(['target'], axis=0)['target'].sort_values().plot(kind='barh', figsize=(12, 8))

In [ ]:
pd.Series(imp_ols_arr, index=X.columns).sort_values().plot(kind='barh', figsize=(12, 8))

In [9]:
def feature_importances(dataset, method_fi='RandomForest'):
    
    Y = dataset['target']
    X = dataset.drop(['target'], axis=1)
    
    if method_fi == 'RandomForest':
        from sklearn.ensemble import RandomForestRegressor
        import numpy
        numpy.random.seed(0)
        result_rf = RandomForestRegressor(n_estimators=10)
        result_rf.fit(X,Y)
        feature_imp = pd.Series(result_rf.feature_importances_, index=X.columns)
    elif method_fi == 'Linear':
        import statsmodels.regression.linear_model
        import math
        result_ols = statsmodels.regression.linear_model.OLS(Y, X).fit()
        imp_ols = result_ols.params
        feature_imp = imp_ols/math.sqrt(sum(imp_ols*imp_ols))
    elif method_fi == 'Correlation':
        feature_imp = dataset.corr().drop(['target'], axis=0)['target']
        
    return feature_imp

In [10]:
feature_importances(data, 'Linear')

Barro-Lee: Percentage Of Population Age 15+ With No Education                                           0.001724
GDP per capita (current US$)                                                                            0.000365
GINI index (World Bank estimate)                                                                       -0.083612
Government expenditure on education, total (% of GDP)                                                   0.111522
Population growth (annual %)                                                                           -0.624336
Share of youth not in education, employment or training, total (% of youth population)                  0.337683
Total alcohol consumption per capita (liters of pure alcohol, projected estimates, 15+ years of age)    0.590149
UIS: Percentage Of Population Age 25+ With No Schooling. Total                                          0.037657
Unemployment, total (% of total labor force) (national estimate)                                